In [ ]:
import jax.numpy as jnp
import jax.scipy.sparse as jsps
import distributions as dist
from jax import jit
import jax
import jax.scipy.stats.multivariate_normal.pdf as mvn_pdf

tau = 0.1
tol = 10e-3
max_iter = 5

In [ ]:
def hamiltionian(q, p):
    return 0.5* (p@Mass_inv@p) - jnp.log(target(q))


gradH_q = jit(jax.grad(hamiltionian, argnums=0))
gradH_p = jit(jax.grad(hamiltionian, argnums=1))
gradH = jit(jax.grad(hamiltionian))
jacH = jit(jax.jacobian(hamiltionian))
jacH_p = jit(jax.jacobian(gradH_p, argnums=1))
jacH_q = jit(jax.jacobian(gradH_q, argnums=0))

print(gradH_p)

# print(jacH_q(q0a, p0b))
# print(jacH_p(q0a, p0b))
jax.jacobian(jax.grad(target))(q0a)
def F(q, p):
        q = q - q0 + tau * gradH_p(0.5*(q+q0), 0.5*(p+p0))
        p = p - p0 - tau * gradH_q(0.5*(q+q0), 0.5*(p+p0))
        return q, p
# jacF_q = jax.jacobian(F, argnums=0)
q0 = q0a
p0 = p0b
tau = 0.1
# qout,pout = jacF_q(q0a,p0b)
q1a, p1b = -F(q0a,p0b)
q2a, p2b = -F(q1a,p1b)
print(q2a, p2b)
# q1a, p1b = F(q2a,p2b)
# print(q1a, p1b)

In [ ]:
newtonq

In [ ]:
# def newton(x0, tol, max_iter):
#     q0, p0 = x0
#     # q0 = q0 + tau *gradH_p
#     @jit
#     def F(q, p):
#         delta_q = q - q0 + tau*gradH_p(0.5*(q + q0), 0.5*(p + p0))
#         delta_p = p - p0 - tau*gradH_q(0.5*(q + q0), 0.5*(p + p0))
#         return delta_q, delta_p
#     jacF_q = jax.jacobian(F, argnums=0)
#     jacF_p = jax.jacobian(F, argnums=1)
#     @jit
#     def update(q, p):
#         # full_jac = jnp.vstack(qout, pout)
#         qout = q0 + jnp.linalg.solve(jacF_q(q0, p0)[0], F(q0, p0)[0])
#         pout = p0 + jnp.linalg.solve(jacF_p(q0, p0)[1], -F(q0, p0)[1])
#         return qout ,pout
#     error = tol+1
#     n = 0
#     while error > tol and n< max_iter:
#         n+=1 
#         qy,py = update(q0, p0)
#         error = jnp.linalg.norm(qy - q0) + jnp.linalg.norm(py- p0)
#         q0 = qy
#         p0 = py
#     return q0, p0
def midpointFPI(q, p):
    q0 = q
    p0 = p
    # q0 = q0 + tau *gradH_p
    @jit
    def F(q, p):
        midq, midp = 0.5*(q+q0), 0.5*(p+p0)
        res_q = q - q0 + tau*gradH_p(midq, midp)
        res_p = p - p0 - tau*gradH_q(midq, midp)
        return res_q, res_p
    @jit
    def newton_step(q, p):
        jacF_q = jax.jacobian(F, argnums=0)(q,p)[0] #dF/dq
        jacF_p = jax.jacobian(F, argnums=1)(q,p)[1] #dF/dp
        F_q, F_p = F(q, p)
        qout = q0 - jnp.linalg.solve(jacF_q, F_q)
        pout = p0 - jnp.linalg.solve(jacF_p, F_p)
        return qout, pout
    @jit
    def conditional(carry):
        q, p, i = carry
        res_q, res_p = F(q, p)
        err = jnp.linagl.norm(res_q) + jnp.linalg.norm(res_p)
        return((err> tol) & (i<max_iter))
    @jit
    def body_step(carry):
        q, p, i = carry
        return newton_step(q, p), i+1
    qout, pout, _ = jax.lax.while_loop(cond, body_step, (q,p,0))
    return qout, pout
x0 = [q0a, p0b]
print(newton(q0a,p0b))
print(q0a, p0b)

In [ ]:
@jit
def leapfrog(q, p):
    def lf_step(carry_in, _):
        q, p = carry_in
        q_half = q + 0.5 * tau * gradH_p(q, p)
        p_new = p - tau * gradH_q(q_half, p)
        q_new = q_half + 0.5 * tau * gradH_p(q_half, p_new)
        carry_out = [q_new, p_new]
        return carry_out, None
    [q_final, p_final], ys = jax.lax.scan(lf_step, [q, p], xs=None, length=T)
    return q_final, p_final

In [ ]:
# mvn_pdf()
target = dist.gauss_ndimf_jax
key = jax.random.PRNGKey(0)
keyq, keyp, key0 = jax.random.split(key, 3)
# q0a = jax.random.normal(keyq, shape=(1,))
# p0b = jax.random.normal(keyp, shape=(1,))
Mass_inv= jnp.eye(len(q0a))
tau = 0.2
T = 1
print(leapfrog(q0a, p0b))
print(q0a, p0b)
# newton([q0a, p0b], tol = 1e-6, max_iter=1000)


In [ ]:
def FPI(xq0, xp0):
    """
    FPI on Hamiltonian system
    Define: 
        tau, 
        H-> (gradH_q, gradH_p)-> (jit_gradH_q, jit_gradH_p)first. 
    """
    def G(y):
        qy, py = y
        qy1 = xq0 + tau*jit_gradH_p(0.5*(qy + xq0), 0.5*(py + xp0))
        py1 = xp0 - tau*jit_gradH_q(0.5*(qy + xq0), 0.5*(py + xp0))
        return  jnp.stack([qy1, py1])
    def F(y):
        y
        return y - G(y)
        
    def conditional(y):
        norm = jnp.linagl.norm(F(y)[0]) + jnp.linalg.norm(F(y)[1])
        return((norm> tol) & (i<max_iter))
    
    
        

In [ ]:
def newton(f, x0, tol, max_iter):
    fprime = jax.grad(f)
    update = lambda x: x-f(x)/fprime(x)
    x = x0
    error = jnp.inf
    n = 0
    while error > tol:
        n +=1
        y = update(x)
        error = abs(x-y)
        x = y
    return x, n
def x_squared(x):
    return x*(1-x)
print(newton(x_squared, 0.7, 1e-5, 100))

In [ ]:
a0 = jnp.array([0.,1.])
a1 = jnp.array([0., 2.])
i = 3
a = jnp.stack([a0, a1])
print(jnp.linalg.norm(a))
# q, p = a
# print(q, p)
# a = jnp.stack([jnp.array([0.,1.]), jnp.array([0.,2.])])
# b = jnp.stack([jnp.array([-1.,1.]), jnp.array([-2.,2.])])
# a-b
print(jnp.linalg.norm(a0)+jnp.linalg.norm(a1))
print(jnp.linalg.norm(a))

In [ ]:
x_squared(2)